In [ ]:
import pandas as pd
import random
seed=random.randint(0,100)
print(seed)

In [ ]:
df=pd.read_csv('../data/label_data_with_cnd/merged_all.csv')
df = df.dropna(subset=["predicted_label"])
df = df[df["predicted_label"].str.strip() != ""]
df['predicted_label'].value_counts()

predicted_label
Valid                 7681
Irrelevant             280
Rant_Without_Visit      22
Advertisement           13
Name: count, dtype: int64

In [ ]:
# Separate the DataFrame into 'valid' and 'non-valid' rows.
df_valid = df[df['predicted_label'] == 'Valid']
df_other = df[df['predicted_label'] != 'Valid']

# Randomly sample 300 rows from the 'valid' DataFrame.
df_valid_sampled = df_valid.sample(n=300, random_state=seed)

# Concatenate the sampled 'valid' rows with the 'non-valid' rows.
df = pd.concat([df_valid_sampled, df_other])

# Print the value counts of the updated DataFrame.
print(df['predicted_label'].value_counts())

predicted_label
Valid                 300
Irrelevant            280
Rant_Without_Visit     22
Advertisement          13
Name: count, dtype: int64


In [4]:
tag_mapping_dict = {
    'Valid': 0,
    'Advertisement': 1,
    'Irrelevant': 2,
    'Rant_Without_Visit': 3,
}

df['label'] = df['predicted_label'].map(tag_mapping_dict)


# REPLACE this:
# df[df['label']==None]
# WITH:
missing = df['label'].isna()
print("Unmapped labels:", int(missing.sum()))
assert missing.sum() == 0, "Found unmapped labels in predicted_label"


Unmapped labels: 0


In [5]:
df_filtered = df.loc[:, ['text', 'label']]

In [6]:
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

/home/zhangguibin/anaconda3/envs/dra_ykm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(
    df_filtered[['text','label']],
    test_size=0.1,
    random_state=seed,
    stratify=df_filtered['label']
)

train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
val_dataset   = Dataset.from_pandas(val_df.reset_index(drop=True))

my_dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset
}).remove_columns([])  # nothing to drop now

print(my_dataset_dict)


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 553
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 62
    })
})


In [8]:
val_df['label'].value_counts()

label
0    30
2    28
3     2
1     2
Name: count, dtype: int64

In [9]:
# gpu usage
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3"

In [10]:
import torch

if torch.cuda.is_available():
    print(f"GPU is available. Device count: {torch.cuda.device_count()}")
    print(f"Current device: {torch.cuda.current_device()}")
    print(f"Device name: {torch.cuda.get_device_name(torch.cuda.current_device())}")
    device = torch.device("cuda")
else:
    print("No GPU available, using CPU.")
    device = torch.device("cpu")

GPU is available. Device count: 3
Current device: 0
Device name: NVIDIA L40S


In [11]:
from transformers import AutoTokenizer, DataCollatorWithPadding

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

INSTRUCTION = (
    "Task: classify review for THIS business. "
    "Labels: Valid, Advertisement, Irrelevant, Rant_Without_Visit. "
    "Priority: Ad > Irrelevant > No-visit rant > Valid."
)

def tokenize_function(example):
    texts = example["text"]
    # texts is a list when batched=True; replicate the instruction to match length
    instr = [INSTRUCTION] * len(texts)
    return tokenizer(instr, texts, truncation=True, max_length=256)


tokenized_datasets = my_dataset_dict.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# drop any raw columns that may exist (prompted_text OR text, and any index col)
cols_to_drop = [c for c in tokenized_datasets["train"].column_names 
                if c in ["prompted_text", "text", "__index_level_0__"]]
if cols_to_drop:
    tokenized_datasets = tokenized_datasets.remove_columns(cols_to_drop)



Map: 100%|██████████| 62/62 [00:00<00:00, 4693.48 examples/s]


In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=4)

# 简单评估指标（最小改动）
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1_weighted": f1_score(labels, preds, average="weighted", zero_division=0),
        "f1_macro": f1_score(labels, preds, average="macro", zero_division=0),
    }

# 2. 设置训练参数
training_args = TrainingArguments(
    output_dir=f"./results/{checkpoint}",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_ratio=0.1,
    weight_decay=0.01,
    logging_dir="./logs",
    # Add these two lines to show training loss
    logging_strategy="steps",
    logging_steps=1,  # Log every 50 steps, you can adjust this value
    fp16=True,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,
)

# 假设你已经准备好了 tokenized_datasets
# tokenized_datasets['train'], tokenized_datasets['test']

# 3. 初始化 Trainer
trainer = Trainer(
    model=model,                                   # 模型会被自动移动到GPU
    args=training_args,                            # 训练参数
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,                           # （替换 processing_class）
    compute_metrics=compute_metrics,               # 增加评估指标
)

# 4. 开始训练
#    在调用 .train() 时，每一批数据也会被自动发送到GPU
trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/zhangguibin/anaconda3/envs/dra_ykm/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Weighted,F1 Macro
1,No log,0.853379,0.741935,0.714959,0.382623
2,1.054700,0.711156,0.774194,0.748394,0.399889
3,0.705300,0.658297,0.774194,0.747149,0.398990


/home/zhangguibin/anaconda3/envs/dra_ykm/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/home/zhangguibin/anaconda3/envs/dra_ykm/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=36, training_loss=0.8478613363371955, metrics={'train_runtime': 11.738, 'train_samples_per_second': 141.336, 'train_steps_per_second': 3.067, 'total_flos': 160512959410752.0, 'train_loss': 0.8478613363371955, 'epoch': 3.0})

In [13]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

id2label = {0: "Valid", 1: "Advertisement", 2: "Irrelevant", 3: "Rant_Without_Visit"}
all_labels = [0, 1, 2, 3]
target_names = [id2label[i] for i in all_labels]

predictions = trainer.predict(tokenized_datasets["validation"])
logits = predictions.predictions
labels = predictions.label_ids
preds = np.argmax(logits, axis=-1).astype(int)

print(classification_report(
    labels, preds,
    labels=all_labels,              # <-- ensure fixed label set
    target_names=target_names,
    zero_division=0,
    digits=3
))

print("Confusion matrix:\n", confusion_matrix(labels, preds, labels=all_labels))


/home/zhangguibin/anaconda3/envs/dra_ykm/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


                    precision    recall  f1-score   support

             Valid      0.765     0.867     0.812        30
     Advertisement      0.000     0.000     0.000         2
        Irrelevant      0.857     0.857     0.857        28
Rant_Without_Visit      0.000     0.000     0.000         2

          accuracy                          0.806        62
         macro avg      0.405     0.431     0.417        62
      weighted avg      0.757     0.806     0.780        62

Confusion matrix:
 [[26  0  4  0]
 [ 2  0  0  0]
 [ 4  0 24  0]
 [ 2  0  0  0]]
